In [3]:
def batch_id_generator(start=0, batch_size=100, num_batches=5):
    """
    Yields lists of IDs starting from 0.
    """
    current_start = start
    for _ in range(num_batches):
        # Generate the range for the current batch
        # list() converts the range object into an actual list of numbers
        yield list(range(current_start, current_start + batch_size))

        # Increment the starting point for the next batch
        current_start += batch_size

In [ ]:
from get_users import get_users
from db_api import put_users
import time

In [4]:
for batch_ids in batch_id_generator(start=150, batch_size=100, num_batches=3000):
    users = get_users(batch_ids)
    put_users(users)
    time.sleep(1)  # peppy will get angry :(

2026-01-07 09:13:00,223 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-07 09:13:00,224 INFO sqlalchemy.engine.Engine INSERT INTO usersqlmodel (id, avatar_url, country_code, username) VALUES (?, ?, ?, ?)
2026-01-07 09:13:00,225 INFO sqlalchemy.engine.Engine [generated in 0.00071s] [(152, 'https://osu.ppy.sh/images/layout/avatar-guest@2x.png', 'US', 'Aardvark Barber'), (158, 'https://osu.ppy.sh/images/layout/avatar-guest@2x.png', 'US', 'Sonicandtails_old'), (159, 'https://a.ppy.sh/159?1337453215.jpg', 'CA', 'meyou22'), (163, 'https://osu.ppy.sh/images/layout/avatar-guest@2x.png', 'US', 'cagaihb'), (164, 'https://osu.ppy.sh/images/layout/avatar-guest@2x.png', 'US', 'anigamersk'), (167, 'https://osu.ppy.sh/images/layout/avatar-guest@2x.png', 'US', 'krisko6'), (171, 'https://osu.ppy.sh/images/layout/avatar-guest@2x.png', 'BE', 'villadelfia'), (172, 'https://a.ppy.sh/172?1199012417.png', 'MX', 'Xionkaiser')  ... displaying 10 of 17 total bound parameter sets ...  (198, 'https://a.ppy

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: usersqlmodel.id
[SQL: INSERT INTO usersqlmodel (id, avatar_url, country_code, username) VALUES (?, ?, ?, ?)]
[parameters: [(152, 'https://osu.ppy.sh/images/layout/avatar-guest@2x.png', 'US', 'Aardvark Barber'), (158, 'https://osu.ppy.sh/images/layout/avatar-guest@2x.png', 'US', 'Sonicandtails_old'), (159, 'https://a.ppy.sh/159?1337453215.jpg', 'CA', 'meyou22'), (163, 'https://osu.ppy.sh/images/layout/avatar-guest@2x.png', 'US', 'cagaihb'), (164, 'https://osu.ppy.sh/images/layout/avatar-guest@2x.png', 'US', 'anigamersk'), (167, 'https://osu.ppy.sh/images/layout/avatar-guest@2x.png', 'US', 'krisko6'), (171, 'https://osu.ppy.sh/images/layout/avatar-guest@2x.png', 'BE', 'villadelfia'), (172, 'https://a.ppy.sh/172?1199012417.png', 'MX', 'Xionkaiser')  ... displaying 10 of 17 total bound parameter sets ...  (198, 'https://a.ppy.sh/198?1728524930.jpeg', 'US', 'ddrballer123'), (199, 'https://a.ppy.sh/199?1604708584.jpeg', 'ES',